<a href="https://colab.research.google.com/github/s-kachroo/CMU-11785-TCR-CLEAN/blob/main/CMU_11785_CLEAN_Baseline_Step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
run_name = '1028-v2'

In [2]:
!git clone https://github.com/s-kachroo/CMU-11785-TCR-CLEAN.git

Cloning into 'CMU-11785-TCR-CLEAN'...
remote: Enumerating objects: 24151, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 24151 (delta 212), reused 151 (delta 134), pack-reused 23854 (from 1)
Receiving objects: 100% (24151/24151), 172.90 MiB | 11.57 MiB/s, done.
Resolving deltas: 100% (297/297), done.
Updating files: 100% (23651/23651), done.


In [3]:
import os
os.chdir('/content/CMU-11785-TCR-CLEAN/app')
current_path = os.getcwd()
os.listdir(current_path)

['train-supconH.py',
 'results',
 'train-triplet.py',
 'requirements.txt',
 'CLEAN_infer_fasta.py',
 'DEMO.ipynb',
 'src',
 'build.py',
 'data',
 'inference.py',
 'gmm.py']

In [4]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00


In [5]:
from src.CLEAN.utils import compute_esm_distance

embedding_folder_name = 'esm2_t33_650M_' + run_name

train_file_name = 'filtered_train_df_' + run_name
valid_file_name = 'filtered_valid_df_' + run_name
test_file_name = 'filtered_test_df_' + run_name
train_csv_name = train_file_name + '.csv'
valid_csv_name = valid_file_name + '.csv'
test_csv_name = test_file_name + '.csv'

train_csv_from = '/content/CMU-11785-TCR-CLEAN/app/data/' + run_name + '/filtered_train_df.csv'
train_csv_to = '/content/CMU-11785-TCR-CLEAN/app/data/' + train_csv_name
!cp $train_csv_from $train_csv_to

valid_csv_from = '/content/CMU-11785-TCR-CLEAN/app/data/' + run_name + '/filtered_valid_df.csv'
valid_vsc_to = '/content/CMU-11785-TCR-CLEAN/app/data/' + valid_csv_name
!cp $valid_csv_from $valid_vsc_to

test_csv_from = '/content/CMU-11785-TCR-CLEAN/app/data/' + run_name + '/filtered_test_df.csv'
test_csv_to = '/content/CMU-11785-TCR-CLEAN/app/data/' + test_csv_name
!cp $test_csv_from $test_csv_to

compute_esm_distance(train_file_name, embedding_folder_name)

100%|██████████| 192/192 [00:00<00:00, 2775.10it/s]


Calculating distance map, number of unique EC is 192


192it [00:00, 1635.36it/s]


In [6]:
import pickle

pkl_train_file_name = '/content/CMU-11785-TCR-CLEAN/app/data/distance_map/' + train_file_name + '.pkl'
pkl_valid_file_name = '/content/CMU-11785-TCR-CLEAN/app/data/distance_map/' + valid_file_name + '.pkl'
pkl_test_file_name = '/content/CMU-11785-TCR-CLEAN/app/data/distance_map/' + test_file_name + '.pkl'

dist_map = pickle.load(open(pkl_train_file_name, 'rb'))
print(dist_map)

{'FLKEKGGL': {'FLKEKGGL': 0.0, 'ILKEPVHGV': 0.7696164, 'EIYKRWII': 0.36661366, 'QVPLRPMTYK': 0.3380317, 'KRWIILGLNK': 0.26384056, 'KAFSPEVIPMF': 0.56136715, 'NEGVKAAW': 0.32420635, 'LQPFPQPELPYPQPQ': 0.6184126, 'LLQTGIHVRVSQPSL': 0.3457064, 'EHPTFTSQYRIQGKL': 0.37978184, 'PKYVKQNTLKLAT': 0.34225014, 'IVTDFSVIK': 0.25513655, 'ELAGIGILTV': 0.31719974, 'RAKFKQLL': 0.31200412, 'AVFDRKSDAK': 0.32092637, 'GILGFVFTL': 0.3542045, 'SLFNTVATLY': 0.5270926, 'RLRAEAQVK': 0.22028893, 'AYAQKIFKI': 0.2572445, 'LLDFVRFMGV': 0.43100706, 'KLGGALQAK': 0.2674622, 'FLYALALLL': 0.82056576, 'FLASKIGRLV': 0.4736663, 'GLCTLVAML': 0.31453636, 'IPSINVHHY': 0.35952005, 'LLFGYPVYV': 0.24844281, 'QYDPVAALF': 0.44567737, 'RTLNAWVKV': 0.35511306, 'KTWGQYWQV': 0.40721974, 'FLRGRAYGL': 0.43648466, 'TPRVTGGGAM': 0.29344827, 'RPPIFIRRL': 0.36472732, 'VTEHDTLLY': 0.20830412, 'QASQEVKNW': 0.36956537, 'TPQDLNTML': 0.40239874, 'YSEHPTFTSQY': 0.33226866, 'HPVGEADYFEY': 0.48019072, 'KRWIIMGLNK': 0.34535822, 'NLVPMVATV': 0.2338

In [7]:
!git pull origin main

From https://github.com/s-kachroo/CMU-11785-TCR-CLEAN
 * branch              main       -> FETCH_HEAD
Already up to date.


In [8]:
!python train-triplet.py --training_data {train_file_name} --model_name {embedding_folder_name} --dir_name {embedding_folder_name} --epoch 500 --batch_size 6000

==> device used: cuda:0 | dtype used:  torch.float32 
==> args: Namespace(learning_rate=0.0005, epoch=500, model_name='esm2_t33_650M_1028-v2', training_data='filtered_train_df_1028-v2', hidden_dim=512, out_dim=128, dir_name='esm2_t33_650M_1028-v2', batch_size=6000, adaptive_rate=100, verbose=False)
Mining hard negatives:
100% 192/192 [00:00<00:00, 6410.04it/s]
The number of unique EC numbers:  192
---------------------------------------------------------------------------
| end of epoch   1 | time:  1.49s | training loss 1.0017
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   2 | time:  0.13s | training loss 0.9622
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   3 | time:  0.13s | training loss 1.0192
------------------------------------------------

In [9]:
!git pull origin main

From https://github.com/s-kachroo/CMU-11785-TCR-CLEAN
 * branch              main       -> FETCH_HEAD
Already up to date.


In [10]:
from src.CLEAN.infer import infer_maxsep
infer_maxsep(train_data=train_file_name, test_data=test_file_name, report_metrics=True, pretrained=False, model_name=embedding_folder_name, dir_name=embedding_folder_name)

# Cleanup files
os.remove(train_csv_to)
os.remove(valid_vsc_to)
os.remove(test_csv_to)

The embedding sizes for train and test: torch.Size([16532, 128]) torch.Size([4680, 128])


100%|██████████| 192/192 [00:00<00:00, 17892.91it/s]


Calculating eval distance map, between 4680 test ids and 192 train EC cluster centers


4680it [00:00, 6312.44it/s]


############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 4680 | total ec: 191 
>>> precision: 0.00485 | recall: 0.00812| F1: 0.00434 | AUC: 0.503 
---------------------------------------------------------------------------
